#  Customer Behavior Project - EDA & Data Cleaning

**Dataset:** Online Retail [1] 
**Objective:** Perform initial exploration and data cleaning to prepare for RFM analysis, clustering, and classification.

- Author: Tarık Buğra Ay
- Date: May, 2025

[1] D. Chen. "Online Retail," UCI Machine Learning Repository, 2015. [Online]. Available: https://doi.org/10.24432/C5BW33.


# Imports

In [2]:
# Core libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings for visuals
%matplotlib inline
sns.set_theme(style='whitegrid')

# Suppress scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Check environment
import sys
print(f"Python version: {sys.version}")


Python version: 3.11.2 (tags/v3.11.2:878ead1, Feb  7 2023, 16:38:35) [MSC v.1934 64 bit (AMD64)]
